# Pandas高级数据处理

In [1]:
import numpy as np
import pandas as pd

## 级联

汇总全年销售数据

In [13]:
df1 = pd.DataFrame(data=[[1,2,3]], columns=list('ABC'))
df1

,A,B,C
0,1,2,3


In [14]:
df2 = pd.DataFrame(data=[[2,3,4]], columns=list('ABC'))
df2

,A,B,C
0,2,3,4


In [18]:
# 级联语法的核心就是索引对齐
# 级联的应用场景：不同期，但是结构相同的数据汇总
# object 对象 array-like 可以是列表 ，也可以是元组
# axis = 0 列索引对齐， axis=1 行索引对齐
res1 = pd.concat([df1, df2], axis=0)

In [19]:
res2 = pd.concat([df1, df2], axis=1)

In [21]:
res1.loc[0]

,A,B,C
0,1,2,3
0,2,3,4


In [23]:
res2.loc[:,'A']

,A,A
0,1,2


In [26]:
# 校验级联之后是否有重复索引
pd.concat((df1, df2), verify_integrity=True)

ValueError: Indexes have overlapping values: Int64Index([0], dtype='int64')

In [28]:
pd.concat((df1, df2), axis=1, verify_integrity=True)

ValueError: Indexes have overlapping values: Index(['A', 'B', 'C'], dtype='object')

In [ ]:
上半年 A
       B
       C

下半年 A
       B
       C

In [31]:
# 通过多层级索引来处理重复索引的问题
pd.concat((df1, df2), axis=1, keys=['上半年','下半年'],names=['周期','产品'])

周期 上半年       下半年      
产品   A  B  C   A  B  C
0    1  2  3   2  3  4

In [36]:
# 通过忽略索引的方式来处理重复索引的问题
pd.concat((df1, df2), ignore_index=True)

,A,B,C
0,1,2,3
1,2,3,4


In [48]:
df3 = pd.DataFrame(data=[[1,2,3,4]], columns=list('CDAB'))
df3

,C,D,A,B
0,1,2,3,4


In [49]:
df2

,A,B,C
0,2,3,4


In [50]:
pd.concat((df3, df2), sort=False)

,C,D,A,B
0,1,2.0,3,4
0,4,NaN,2,3


In [ ]:
pd.concat()

In [51]:
df4 = pd.DataFrame(data=np.random.randint(0, 100, size=(3,4)), columns=list('ABCD'))
df5 = pd.DataFrame(data=np.random.randint(-100, 0, size=(4,3)), columns=list('BDE'))

In [52]:
display(df4, df5)

,A,B,C,D
0,82,20,46,14
1,51,14,31,47
2,23,18,47,38


,B,D,E
0,-18,-1,-5
1,-32,-19,-73
2,-63,-20,-73
3,-15,-45,-61


In [58]:
df1

,A,B,C
0,1,2,3


In [59]:
df4

,A,B,C,D
0,82,20,46,14
1,51,14,31,47
2,23,18,47,38


In [60]:
df5

,B,D,E
0,-18,-1,-5
1,-32,-19,-73
2,-63,-20,-73
3,-15,-45,-61


In [63]:
# outer 保留级联方向的所有【标签】 并集
# innert 保留级联方向的共有【标签】 交集
pd.concat((df4, df5, df1), sort=True, join='outer')

,A,B,C,D,E
0,82.0,20,46.0,14.0,NaN
1,51.0,14,31.0,47.0,NaN
2,23.0,18,47.0,38.0,NaN
0,NaN,-18,NaN,-1.0,-5.0
1,NaN,-32,NaN,-19.0,-73.0
2,NaN,-63,NaN,-20.0,-73.0
3,NaN,-15,NaN,-45.0,-61.0
0,1.0,2,3.0,NaN,NaN


## 合并

合并就是根据两张表的公共信息，把两张表的数据汇总的方法。

合并以列的内容为参考标准，不存在行合并，都是列合并

合并的列通常是离散型数据。可以是数值型

合并的列之间存在一对一、一对多、多对多关系，否则合并结果为空

### 计算上半年订单总额GMV

参数： left_on, right_on

In [7]:
first_half_year = pd.read_excel('合并表格案例.xlsx', sheet_name=0)
second_half_year = pd.read_excel('合并表格案例.xlsx', sheet_name=1)

display(first_half_year.head(), second_half_year.head())

,用户ID,商品ID,订单ID,购买数量
0,lucy,10001,1009,1
1,jack,10002,1002,2
2,lucy,10003,1007,1
3,alex,10004,1010,1
4,mery,10005,1008,1


,用户ID,商品ID,订单ID,购买数量
0,tom,10006,2001,1
1,oldshang,10003,2002,1
2,佩奇,10004,2003,1
3,小明,10004,2004,2
4,小红,10001,2005,1


In [4]:
user_table = pd.read_excel('合并表格案例.xlsx', sheet_name=2)
user_table.head()

,用户ID,地区,VIP等级,手机号
0,lucy,北京,3,13054344433
1,智哥,深圳,3,13046798795
2,mery,北京,2,17877659878
3,jack,上海,4,18635482221
4,alex,北京,1,17601002323


In [5]:
product_table = pd.read_excel('合并表格案例.xlsx', sheet_name=3)
product_table.head()

,商品ID,商品类别,商品品牌,商品单价
0,10001,笔记本,华为,8000
1,10002,笔记本,小米,7600
2,10003,鼠标,华为,300
3,10004,鼠标,apple,600
4,10005,键盘,apple,1000


In [6]:
return_table = pd.read_excel('合并表格案例.xlsx', sheet_name=4)
return_table.head()

,订单_id,退货状态
0,1003,退货中
1,1004,退货中
2,1005,退货完成
3,1011,退货完成
4,1014,退货中


In [14]:
# 两张表合并时，默认是根据所有的相同字段名称的列来进行合并
res1 = pd.merge(left=first_half_year, right=product_table)
res1['订单总额'] = res1['购买数量'] * res1['商品单价']
res1['订单总额'].sum()

79500

### 获取上半年用户地区，查看各地区订单数量

参数: how

In [20]:
user_table

,用户ID,地区,VIP等级,手机号
0,lucy,北京,3,13054344433
1,智哥,深圳,3,13046798795
2,mery,北京,2,17877659878
3,jack,上海,4,18635482221
4,alex,北京,1,17601002323
5,tom,深圳,2,18910538799
6,oldshang,北京,3,17699887678
7,佩奇,上海,2,15600140101
8,小明,上海,2,18789897788
9,小红,北京,3,17625745653


In [21]:
first_half_year

,用户ID,商品ID,订单ID,购买数量
0,lucy,10001,1009,1
1,jack,10002,1002,2
2,lucy,10003,1007,1
3,alex,10004,1010,1
4,mery,10005,1008,1
5,jack,10006,1003,1
6,佩奇,10002,1013,1
7,alex,10002,1001,3
8,智哥,10004,1004,1
9,tom,10001,1011,1


In [28]:
# 'left', 'right', 'outer', 'inner' 基于列的内容
#  inner  只保留合并列内容的交集
#  outer  保留合并列内容的并集
#  left   保留左表拥有的内容
#  right  保留右表拥有的内容
pd.merge(left=first_half_year, right=user_table, how='inner')['地区'].value_counts()

北京    6
上海    3
深圳    2
Name: 地区, dtype: int64

### 找出上半年和下半年购买过相同商品的用户

参数解释：on, suffixes

一个用户，上半年和下半年都购买了同一个商品

In [29]:
first_half_year

,用户ID,商品ID,订单ID,购买数量
0,lucy,10001,1009,1
1,jack,10002,1002,2
2,lucy,10003,1007,1
3,alex,10004,1010,1
4,mery,10005,1008,1
5,jack,10006,1003,1
6,佩奇,10002,1013,1
7,alex,10002,1001,3
8,智哥,10004,1004,1
9,tom,10001,1011,1


In [30]:
second_half_year

,用户ID,商品ID,订单ID,购买数量
0,tom,10006,2001,1
1,oldshang,10003,2002,1
2,佩奇,10004,2003,1
3,小明,10004,2004,2
4,小红,10001,2005,1
5,大王,10006,2006,1
6,lucy,10003,2007,1
7,佩奇,10002,2008,1
8,小明,10002,2009,2
9,mery,10005,2010,1


In [33]:
# on 多个相同名称的字段中，指定合并要参考的列标签
# suffixes 为其他相同名称的字段且未参加合并的列添加后缀名称
pd.merge(left=first_half_year, right=second_half_year, 
         on=['用户ID','商品ID'], 
         suffixes=['_上半年','_下半年'])

,用户ID,商品ID,订单ID_上半年,购买数量_上半年,订单ID_下半年,购买数量_下半年
0,lucy,10003,1007,1,2007,1
1,mery,10005,1008,1,2010,1
2,佩奇,10002,1013,1,2008,1
3,智哥,10004,1004,1,2011,3


### 查看上半年退货商品总额

参数：left_on, right_on, left_index, right_index

In [34]:
return_table

,订单_id,退货状态
0,1003,退货中
1,1004,退货中
2,1005,退货完成
3,1011,退货完成
4,1014,退货中
5,1007,退货完成


In [37]:
first_half_year

,用户ID,商品ID,订单ID,购买数量
0,lucy,10001,1009,1
1,jack,10002,1002,2
2,lucy,10003,1007,1
3,alex,10004,1010,1
4,mery,10005,1008,1
5,jack,10006,1003,1
6,佩奇,10002,1013,1
7,alex,10002,1001,3
8,智哥,10004,1004,1
9,tom,10001,1011,1


In [39]:
res2 = pd.merge(left=first_half_year, right=return_table, left_on='订单ID', right_on='订单_id')

In [40]:
res2

,用户ID,商品ID,订单ID,购买数量,订单_id,退货状态
0,lucy,10003,1007,1,1007,退货完成
1,jack,10006,1003,1,1003,退货中
2,智哥,10004,1004,1,1004,退货中
3,tom,10001,1011,1,1011,退货完成


In [49]:
res3 = pd.merge(left=res2, right=product_table, how='inner')
(res3['购买数量']*res3['商品单价']).sum()

9100

### 汇总全年订单数据

In [52]:
total = pd.concat((first_half_year, second_half_year), ignore_index=True)

### 计算全年客单价

每一单成交金额均值

In [56]:
res4 = pd.merge(left=total, right=product_table)
(res4['购买数量'] * res4['商品单价']).mean()

5126.086956521739

In [ ]:
merge

left\right 
on\suffiexes
left_on\right_on 
how: inner outer left right
    
left_index\right_index: 当左右表需要使用行索引作为合并列时，True

## 分组

分组必聚合

In [59]:
df = pd.read_excel('分组表格案例.xlsx', index_col=0)

In [60]:
df

,菜品,颜色,价格,数量
0,白菜,绿,64,88
1,白菜,红,24,98
2,冬瓜,红,0,43
3,辣椒,红,59,51
4,西红柿,白,29,29
5,白菜,白,5,37
6,冬瓜,红,41,36
7,冬瓜,白,43,24
8,冬瓜,白,88,8
9,辣椒,红,61,28


groupby()

In [67]:
df.head()

,菜品,颜色,价格,数量
0,白菜,绿,64,88
1,白菜,红,24,98
2,冬瓜,红,0,43
3,辣椒,红,59,51
4,西红柿,白,29,29


In [74]:
# DataFrameGroupBy 对象可以直接响应pandas的聚合函数 
# 注意：聚合方法只能对可运算类型有效
df.groupby(by=['菜品'])['数量'].sum()

菜品
冬瓜     220
白菜     310
西红柿    229
辣椒     573
Name: 数量, dtype: int64

In [71]:
df.groupby(by=['菜品'])['价格'].mean()

菜品
冬瓜     53.000
白菜     34.375
西红柿    18.000
辣椒     41.000
Name: 价格, dtype: float64

groups

可以用于查看DataFrameGroupBy所表达的分组情况

In [62]:
df.groupby(by=['菜品']).groups

{'冬瓜': Int64Index([2, 6, 7, 8, 21, 28], dtype='int64'),
 '白菜': Int64Index([0, 1, 5, 13, 17, 25, 26, 27], dtype='int64'),
 '西红柿': Int64Index([4, 10, 14, 22, 29], dtype='int64'),
 '辣椒': Int64Index([3, 9, 11, 12, 15, 16, 18, 19, 20, 23, 24], dtype='int64')}

In [64]:
df.loc[[0, 1, 5, 13, 17, 25, 26, 27]]

,菜品,颜色,价格,数量
0,白菜,绿,64,88
1,白菜,红,24,98
5,白菜,白,5,37
13,白菜,红,66,11
17,白菜,白,10,39
25,白菜,白,70,26
26,白菜,白,10,2
27,白菜,白,26,9


多分组

In [83]:
df.groupby(by=['菜品','颜色'])['价格'].mean()

菜品   颜色
冬瓜   白     67.666667
     红     38.333333
白菜   白     24.200000
     红     45.000000
     绿     64.000000
西红柿  白     25.500000
     红     12.000000
     绿     13.500000
辣椒   白     40.200000
     红     41.666667
Name: 价格, dtype: float64

In [84]:
df.groupby(by=['颜色','菜品'])['数量'].sum()

颜色  菜品 
白   冬瓜     118
    白菜     113
    西红柿     87
    辣椒     244
红   冬瓜     102
    白菜     109
    西红柿     49
    辣椒     329
绿   白菜      88
    西红柿     93
Name: 数量, dtype: int64

定制多种聚合指标

In [78]:
gpobj =  df.groupby('菜品')

# agg 接收一个字典对象
# 字典对象里：键是要聚合的列名称， 值是一个函数名字（地址），是一个聚合函数
gpobj.agg({
    '价格':np.mean,
    '数量':np.sum
})

,价格,数量
菜品,,
冬瓜,53.000,220
白菜,34.375,310
西红柿,18.000,229
辣椒,41.000,573


高级聚合

In [86]:
# df.groupby('菜品')['价格'].mean()

df.groupby('菜品')['价格'].apply(np.mean)

菜品
冬瓜     53.000
白菜     34.375
西红柿    18.000
辣椒     41.000
Name: 价格, dtype: float64

In [89]:
def my_mean(x):
    # 对于分组对象而言，apply传递的函数，接收到的参数x是一组数据
    return x.mean()

In [90]:
df.groupby('菜品')['价格'].apply(my_mean)

菜品
冬瓜     53.000
白菜     34.375
西红柿    18.000
辣椒     41.000
Name: 价格, dtype: float64

## 交叉表

交叉表统计的数量 count

In [93]:
df.groupby(['菜品','颜色'])['数量'].count().unstack()

颜色,白,红,绿
菜品,,,
冬瓜,3.0,3.0,NaN
白菜,5.0,2.0,1.0
西红柿,2.0,1.0,2.0
辣椒,5.0,6.0,NaN


In [94]:
# index  是一个序列，而不是一个列标签
# columns 
pd.crosstab(index=df['菜品'], columns=df['颜色'])

颜色,白,红,绿
菜品,,,
冬瓜,3,3,0
白菜,5,2,1
西红柿,2,1,2
辣椒,5,6,0


## 透视表

In [96]:
# data 数据源 就是要进行透视的DataFrame对象
# index\columns 透视表的行列是从数据源的哪列提取的
# values 是要统计的数据源中的字段
# aggfunc 是聚合方法，传递的是函数名字
pd.pivot_table(data=df, index='菜品', columns='颜色', values=['价格'], aggfunc=np.mean)

价格                 
颜色           白          红     绿
菜品                             
冬瓜   67.666667  38.333333   NaN
白菜   24.200000  45.000000  64.0
西红柿  25.500000  12.000000  13.5
辣椒   40.200000  41.666667   NaN